# Reliability Analysis

In [13]:
from dataclasses import dataclass
import numpy as np
from scipy.special import expit

1) What is p?

2) The Data looks like this.

In [10]:
@dataclass
class Observation:
    broke: bool
    days: int # include the day it broke as well
        
    def __str__(self):
        return f'{self.days} {self.broke}'
        
def gen_data(p, max_n): # p is survival prob
    for i in range(1, max_n+1):
        
        if np.random.random() > p:
            return Observation(broke=True, days=i)
    return Observation(broke=False, days=max_n)

np.random.seed(555)
p = 1-0.0123 # pretend you didn't see this
data = [gen_data(p, 100) for i in range(200)]

3) Try Plot PDF P(break on day d)

4) How to we fix the fact the the graph is very spotty? What is the better thing to plot?

5) If you just average the day it breaks what happen?

6) Why is regression on time to fail a bad idea?

7) How to find p? Find it. Find p along with MTBF.

8) Wear and Tear and other features. How do we incoporate wear and tear in our model?

9) Find w

In [41]:
@dataclass
class Observation:
    broke: bool
    days: int # include the day it broke as well
        
    def __str__(self):
        return f'{self.days} {self.broke}'

def p(t):
    return expit(-t*0.015 +4) # again pretend you didn't see this

def gen_data(p, max_n): # p is survival prob
    for i in range(1, max_n+1):
        if np.random.random() > p(i):
            return Observation(broke=True, days=i)
    return Observation(broke=False, days=max_n)

np.random.seed(555)
data = [gen_data(p, 100) for i in range(200)]

10) What is MTBF now?

11) What is the p for day = 30?